In [38]:
from transformers import Wav2Vec2CTCTokenizer
from transformers import SeamlessM4TFeatureExtractor
from malaya_speech.utils.char import HF_CTC_VOCAB
from conformer import ConformerConfig, ConformerEncoder
import malaya_speech
import json

In [8]:
HF_CTC_VOCAB_INDEX = {c: no for no, c in enumerate(HF_CTC_VOCAB)}
HF_CTC_VOCAB_INDEX['|'] = HF_CTC_VOCAB_INDEX[' ']
del HF_CTC_VOCAB_INDEX[' ']
HF_CTC_VOCAB_INDEX['[UNK]'] = len(HF_CTC_VOCAB_INDEX)
HF_CTC_VOCAB_INDEX['[PAD]'] = len(HF_CTC_VOCAB_INDEX)

In [10]:
with open('vocab.json', 'w') as vocab_file:
    json.dump(HF_CTC_VOCAB_INDEX, vocab_file)

In [11]:
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


In [40]:
tokenizer

Wav2Vec2CTCTokenizer(name_or_path='./', vocab_size=41, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	39: AddedToken("[UNK]", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	40: AddedToken("[PAD]", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	41: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	42: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

In [39]:
tokenizer.pad_token_id

40

In [20]:
label_features = tokenizer(['asd', 'as'])
label_features

{'input_ids': [[1, 19, 4], [1, 19]], 'attention_mask': [[1, 1, 1], [1, 1]]}

In [24]:
labels_batch = tokenizer.pad(
    label_features,
    padding='longest',
    return_tensors="pt",
)

# replace padding with -100 to ignore loss correctly
labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

In [25]:
labels

tensor([[   1,   19,    4],
        [   1,   19, -100]])

In [27]:
feature_extractor = SeamlessM4TFeatureExtractor.from_pretrained("facebook/w2v-bert-2.0")

preprocessor_config.json:   0%|          | 0.00/275 [00:00<?, ?B/s]

In [28]:
from transformers import Wav2Vec2BertProcessor

processor = Wav2Vec2BertProcessor(feature_extractor=feature_extractor, tokenizer=tokenizer)

ImportError: cannot import name 'Wav2Vec2BertProcessor' from 'transformers' (/home/husein/.local/lib/python3.8/site-packages/transformers/__init__.py)

In [42]:
y, sr = malaya_speech.load('/home/husein/dev/malaya-speech/speech/example-speaker/husein-zolkepli.wav')
y2, sr = malaya_speech.load('/home/husein/dev/malaya-speech/speech/example-speaker/shafiqah-idayu.wav')

In [43]:
feature_extractor([y, y2])['input_features']

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


array([[[-4.1859403 , -4.3956175 , -2.0071175 , ..., -0.8214691 ,
         -0.5817979 , -0.31279954],
        [-1.6673652 , -1.944707  , -1.4485894 , ..., -0.09358254,
         -0.22961655, -0.11838696],
        [-1.1028099 , -0.8061468 , -1.362634  , ..., -0.25970015,
         -0.33983642, -0.15761521],
        ...,
        [-0.8000384 , -1.3038509 , -0.43857166, ..., -1.1498504 ,
         -1.0206455 , -0.5446591 ],
        [ 0.36477318,  0.28267542, -0.5236914 , ..., -0.7319313 ,
         -0.42760292, -0.56898147],
        [ 0.7143059 ,  0.44851705, -0.3786471 , ...,  1.        ,
          1.        ,  1.        ]],

       [[-1.3710316 , -0.36263075,  0.17935964, ...,  0.6701742 ,
          0.48520747,  0.6884587 ],
        [ 0.06079479,  0.35314864,  0.06038192, ..., -0.04796737,
          0.287562  ,  0.02813615],
        [-1.4509171 , -0.49308303,  0.39764035, ..., -0.3195095 ,
         -0.3137719 ,  0.03174698],
        ...,
        [ 1.        ,  1.        ,  1.        , ...,  

In [ ]:
config = ConformerConfig(
    input_dim=80,
    output_dim=len(HF_CTC_VOCAB),
    time_reduction_stride=4,
    conformer_input_dim=144,
    conformer_ffn_dim=576,
    conformer_num_layers=8,
    conformer_num_heads=4,
    conformer_depthwise_conv_kernel_size=31,
    conformer_dropout=0.1,
    pad_token_id=len(HF_CTC_VOCAB) - 1,
    ctc_loss_reduction='mean',
    ctc_zero_infinity=True,
)